In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import keras

In [ ]:
import os
print( os.listdir('/content') )

['datalab', '.rnd', '.config', 'model_only.h5', '.local', '.ipython', 'Steel', 'model_with_weights.h5', '.nv', '.forever', 'data_cls.zip', '.keras', '.cache']


In [ ]:
from keras import initializers
from keras.layers import BatchNormalization
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [ ]:
tf.set_random_seed(42)

In [ ]:
classifier = Sequential()
 
# Step 1 - Convolution
classifier.add(Conv2D(64, (5, 5), input_shape = (200, 200, 3), activation = 'relu', kernel_initializer=initializers.uniform(seed=42)))
 
# Step 2 - Pooling (first Batch normalization)
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (4, 4)))
 
# Adding a second convolutional layer
 
classifier.add(Conv2D(64, (4, 4), activation = 'relu', kernel_initializer=initializers.uniform(seed=42)))
 
classifier.add(BatchNormalization())
 
classifier.add(MaxPooling2D(pool_size = (2, 2)))
 
# Adding a third convolutional layer
 
classifier.add(Conv2D(32, (4, 4), activation = 'relu', kernel_initializer=initializers.uniform(seed=42)))
 
classifier.add(BatchNormalization())
 
classifier.add(MaxPooling2D(pool_size = (2, 2)))
 
# Adding a fourth convolutional layer
 
classifier.add(Conv2D(32, (4, 4), activation = 'relu', kernel_initializer=initializers.uniform(seed=42)))
 
classifier.add(BatchNormalization())
 
classifier.add(MaxPooling2D(pool_size = (2, 2)))
 
# Step 3 - Flattening
classifier.add(Flatten())
 
# Step 4 - Full connection 1
 
classifier.add(Dense(units = 4096, activation = 'relu'))
 
classifier.add(BatchNormalization())
 
classifier.add(Dropout(rate=0.3, seed=42))
 
# Step 6 - Full connection 2
 
classifier.add(Dense(units = 1024, activation = 'relu'))
 
classifier.add(BatchNormalization())
 
classifier.add(Dense(units = 6, activation = 'softmax'))

In [ ]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 196, 196, 64)      4864      
_________________________________________________________________
batch_normalization_1 (Batch (None, 196, 196, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        65600     
_________________________________________________________________
batch_normalization_2 (Batch (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 32)        32800     
__________

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data_cls.zip to data_cls.zip


In [ ]:
!mkdir Steel && unzip data_cls.zip -d Steel/

Archive:  data_cls.zip
   creating: Steel/data_cls/
   creating: Steel/data_cls/test/
   creating: Steel/data_cls/test/crazing/
  inflating: Steel/data_cls/test/crazing/Cr_100.bmp  
  inflating: Steel/data_cls/test/crazing/Cr_101.bmp  
  inflating: Steel/data_cls/test/crazing/Cr_103.bmp  
  inflating: Steel/data_cls/test/crazing/Cr_105.bmp  
  inflating: Steel/data_cls/test/crazing/Cr_106.bmp  
  inflating: Steel/data_cls/test/crazing/Cr_107.bmp  
  inflating: Steel/data_cls/test/crazing/Cr_108.bmp  
  inflating: Steel/data_cls/test/crazing/Cr_109.bmp  
  inflating: Steel/data_cls/test/crazing/Cr_110.bmp  
  inflating: Steel/data_cls/test/crazing/Cr_111.bmp  
  inflating: Steel/data_cls/test/crazing/Cr_112.bmp  
  inflating: Steel/data_cls/test/crazing/Cr_113.bmp  
  inflating: Steel/data_cls/test/crazing/Cr_114.bmp  
  inflating: Steel/data_cls/test/crazing/Cr_115.bmp  
  inflating: Steel/data_cls/test/crazing/Cr_116.bmp  
  inflating: Steel/data_cls/test/crazing/Cr_117.bmp  
  inflat

In [ ]:
train_data_dir ='Steel/data_cls/train/'
validation_data_dir = 'Steel/data_cls/validation'
test_data_dir = 'Steel/data_cls/test/'

In [ ]:
# Augmentation configuration Of images
print("augmentation configuration for training and testing")
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   zoom_range=0.2,
                                   vertical_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
print("augmentation configuration completed")

augmentation configuration for training and testing
augmentation configuration completed


In [ ]:
# Generating Dataset
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(200,200),
        batch_size=64,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(200, 200),
        batch_size=64,
        class_mode='binary')

Found 1500 images belonging to 6 classes.
Found 300 images belonging to 6 classes.


In [ ]:
optimizer_adam = adam(lr = 0.0005, decay = 0.0001)
 
classifier.compile(optimizer = optimizer_adam, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
train = True
nb_test_samples = 300
nb_test_samples = 1500 

In [ ]:
 if(train == True):
    #Using the early stopping technique to prevent overfitting
    earlyStopping= keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, verbose=1, mode='auto')
    print("Fitting the model")
    history = classifier.fit_generator(
            train_generator,
            steps_per_epoch=500,
            callbacks=[earlyStopping],
            epochs=10,
            validation_data=test_generator,
            validation_steps=nb_test_samples)

 #Saving the Model
print("Saving the weights")
classifier.save('model_only.h5')
classifier.save_weights('model_with_weights.h5')

Fitting the model
Epoch 1/10
500/500 [==============================] - 585s 1s/step - loss: 0.3144 - acc: 0.8986 - val_loss: 1.4964 - val_acc: 0.6067
Epoch 2/10
184/500 [==========>...................] - ETA: 3:52 - loss: 0.1648 - acc: 0.9434500/500 [==============================] - 582s 1s/step - loss: 0.1446 - acc: 0.9499 - val_loss: 9.2146 - val_acc: 0.3400
Epoch 3/10
346/500 [===================>..........] - ETA: 1:53 - loss: 0.1235 - acc: 0.9571500/500 [==============================] - 582s 1s/step - loss: 0.1226 - acc: 0.9574 - val_loss: 8.7996 - val_acc: 0.2433
Epoch 4/10
408/500 [=======================>......] - ETA: 1:08 - loss: 0.1012 - acc: 0.9646500/500 [==============================] - 585s 1s/step - loss: 0.1009 - acc: 0.9648 - val_loss: 0.2854 - val_acc: 0.9333
Epoch 5/10
431/500 [========================>.....] - ETA: 50s - loss: 0.0941 - acc: 0.9690500/500 [==============================] - 581s 1s/step - loss: 0.0912 - acc: 0.9701 - val_loss: 2.3685 - val_acc: 0

In [ ]:
IMAGE_WIDTH=200
IMAGE_HEIGHT=200

In [ ]:
## Predicting the result1
from keras.preprocessing import image
test_image = image.load_img('Steel/data_cls/validation/1.bmp', target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
cls = train_generator.class_indices
cls = train_generator.class_indices
pred_cls = np.argmax(result)
cls_name = list(cls.keys())
cls_name[pred_cls]

'patches'

In [ ]:
## Predicting the result2
from keras.preprocessing import image
test_image = image.load_img('Steel/data_cls/validation/2.bmp', target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
cls = train_generator.class_indices
cls = train_generator.class_indices
pred_cls = np.argmax(result)
cls_name = list(cls.keys())
cls_name[pred_cls]

'patches'

In [ ]:
## Predicting the result3
from keras.preprocessing import image
test_image = image.load_img('Steel/data_cls/validation/3.bmp', target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
cls = train_generator.class_indices
cls = train_generator.class_indices
pred_cls = np.argmax(result)
cls_name = list(cls.keys())
cls_name[pred_cls]

'patches'

In [ ]:
## Predicting the result3
from keras.preprocessing import image
test_image = image.load_img('Steel/data_cls/validation/4.bmp', target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
cls = train_generator.class_indices
cls = train_generator.class_indices
pred_cls = np.argmax(result)
cls_name = list(cls.keys())
cls_name[pred_cls]

'patches'

In [ ]:
## Predicting the result5
from keras.preprocessing import image
test_image = image.load_img('Steel/data_cls/validation/5.bmp', target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
cls = train_generator.class_indices
cls = train_generator.class_indices
pred_cls = np.argmax(result)
cls_name = list(cls.keys())
cls_name[pred_cls]

'patches'

In [ ]:
## Predicting the result6
from keras.preprocessing import image
test_image = image.load_img('Steel/data_cls/validation/6.bmp', target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
cls = train_generator.class_indices
cls = train_generator.class_indices
pred_cls = np.argmax(result)
cls_name = list(cls.keys())
cls_name[pred_cls]

'patches'

In [ ]:
test_image = image.load_img('Steel/data_cls/train/crazing/Cr_161.bmp', target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
cls = train_generator.class_indices
cls = train_generator.class_indices
pred_cls = np.argmax(result)
cls_name = list(cls.keys())
cls_name[pred_cls]

'patches'

In [ ]:
test_image = image.load_img('Steel/data_cls/train/crazing/Cr_160.bmp', target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
cls = train_generator.class_indices
cls = train_generator.class_indices
pred_cls = np.argmax(result)
cls_name = list(cls.keys())
# cls_name[pred_cls]
result

array([[0., 0., 1., 0., 0., 0.]], dtype=float32)

In [ ]:
test_image = image.load_img('Steel/data_cls/train/patches/Pa_162.bmp', target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
cls = train_generator.class_indices
cls = train_generator.class_indices
pred_cls = np.argmax(result)
cls_name = list(cls.keys())
cls_name[pred_cls]
result

array([[0., 0., 1., 0., 0., 0.]], dtype=float32)